# KNN POR LIDERES Y SEGUIDORES

In [ ]:
from pyspark import SparkConf, SparkContext

sc.stop()

"""
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

sc = pyspark.SparkContext('local[*]')
"""

conf = (SparkConf()
        .setMaster("local[3]")
        .setAppName("TP Orga")
        .set("spark.executor.cores", "3")        # numero de cores a utilizar
#        .set("spark.executor.memory", "2g")     # cada executor puede correr en maximo 2 task al mismo tiempo
#        .set("spark.executor.instances", "6")    # numero de executors
        .set("spark.dynamicAllocation.enabled", "true") # usar estas dos lineas en vez de executor.instances
        .set("spark.shuffle.service.enabled", "true")

        .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
#        .set("spark.driver.maxResultSize", "2g")
#        .set("spark.cores.max", "2") 
         .set("spark.default.parallelism", "3") )


sc = SparkContext(conf = conf)

#sc = pyspark.SparkContext('local[2]')

print sc._conf.getAll()


# Preprocesamiento del texto

In [ ]:
import re

stop_words = ["a", "about", "above", "after", "all", "am", "an", "and", "any",\
                "are", "as", "at", "be", "because", "been", "before", "being",\
              "below", "tween", "bot", "by", "could", "did", "do", "does", "doing",\
              "down", "during", "each", "few", "for", "from", "further",\
              "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her",\
              "here", "here's", "hers", "herself", "him", "himself", "his", "how",\
              "how's", "i", "i'd", "i'll", "i'm", "i've", "u'if", "in", "into", "is",\
              "it", "it's", "its", "itself", "let's", "me", "more", "most",\
              "my", "myself", "of", "off", "on", "once",\
              "only", "or", "other", "ought", "our", "ours", "ourselves", "out",\
              "over", "own", "same", "she", "she'd", "she'll", "she's",\
              "should", "so", "some", "such", "than", "that","that's",\
              "the", "their", "theirs", "them", "themselves", "then", "there", "there's",\
              "these", "they", "they'd", "they'll", "they're", "they've", "this",\
              "those", "through", "to", "too", "under", "until", "up", "very", "was",\
             "we", "we'd", "we'll", "we're", "we've", "were",\
              "what", "what's", "when", "when's", "where", "where's", "which", "while",\
              "who", "who's", "whom", "why", "why's", "with", "would",\
              "you", "you'd", "you'll", "you're", "you've", "your", "yours",\
              "yourself", "yourselves"]

stop_words = set(stop_words)

palabras_sentimiento_inverso = ["not", "ain't", "bareley", "aren't", "can't", "couldn't", "didn't", "doesn't",\
                                "don't", "hadn't", "few", "hardly", "low", "merely", "neither", "never",\
                                "no", "nor", "nobody", "none", "nope", "nothing", "rarely", "seldom",\
                                "hasn't", "haven't", "isn't", "mustn't", "shan't", "shouldn't", "used",\
                                "wasn't", "weren't", "won't", "wouldn't", "zero"]


palabras_sentimiento_inverso = set(palabras_sentimiento_inverso)



def quitar_reviews_repetidos(reviewsRDD):
    reviews_sin_rep = reviewsRDD.map( lambda review: (review.Text, review) )\
                    .reduceByKey( lambda review1, review2: review1 )

    reviews_sin_rep = reviews_sin_rep.map(lambda x: x[1])

    return reviews_sin_rep

def preprocesar(review):
    texto = quitar_signos(review.Text)
    resumen = quitar_signos(review.Summary)

#    texto = texto.lower()
#    resumen = resumen.lower()

    texto_palabras = texto.split(" ")
    resumen_palabras = resumen.split(" ")

#    texto_palabras = texto_palabras + ngrams(texto_palabras, 2)
#    resumen_palabras = resumen_palabras + ngrams(resumen_palabras, 2)

    texto_palabras = quitar_stopWords(texto_palabras)
    resumen_palabras = quitar_stopWords(resumen_palabras)

    estructura = [review.Id, review.Prediction, resumen_palabras+texto_palabras]

    return estructura


def quitar_signos(texto):
    text = ""
    if texto is not None:
        text = texto.replace("<br />", " ")
        text = re.sub("[^a-zA-Z' ]", ' ', text)

    return text


def quitar_stopWords(texto):
    text = []

    for i in range( len(texto) ):
        palabra = texto[i].lower()
        if palabra not in stop_words and palabra != '':

            if palabra in palabras_sentimiento_inverso and i+1 < len(texto):
                palabra = palabra + " " + texto[i+1].lower()
                text.append(palabra)

            if i > 0 and texto[i-1].lower() not in palabras_sentimiento_inverso:
                text.append(palabra)

    return text


def ngrams(texto, n):
    #texto = texto.split(" ")
    lista_ngrams = []
    for i in xrange(0, len(texto)-n-1):
        lista_ngrams.append(" ".join(texto[i:i+n]))

    return lista_ngrams


# Creacion RDD train

In [ ]:
import pyspark_csv as pycsv
from pyspark.sql import SQLContext

sqlCtx=SQLContext(sc)
num_partition = 3

trainRDD = sc.textFile('data_train.csv', num_partition)
#trainRDD = sc.textFile('train.csv', num_partition)

train_dataframe = pycsv.csvToDataFrame(sqlCtx, trainRDD, parseDate=False)
train = train_dataframe.rdd


train_sin_rep = quitar_reviews_repetidos(train)

train_preprocesado = train_sin_rep.map(preprocesar)

for x in train_preprocesado.take(1):
    for y in x:
        print y

print "Count: ", train_preprocesado.count()


In [ ]:
def bow(review):
    texto = review[2]
    frec = {}
    for word in texto:
        if word in frec:
            frec[word] += 1
        else:
            frec[word] = 1

    review.append(frec)
#    review[2] = frec
    return review

train_preprocesado = train_preprocesado.map(bow)

print "Count: ", train_preprocesado.count()

#train_preprocesado = train_preprocesado.cache()


# Metrica

In [ ]:

def frec_por_doc(review):
    texto = set(review[2])
    words = []
    for word in texto:
        words.append( (word, 1) )

    return words

positivas = train_preprocesado.filter( lambda review: review[1]==4 or review[1]==5 or review[1]==3)
cant_pos = positivas.count()

frec_por_doc_pos = positivas.flatMap(frec_por_doc).reduceByKey( lambda x,y: x+y )
frec_por_doc_pos = frec_por_doc_pos.collect()

frec_pos = {}

for word in frec_por_doc_pos:
    frec_pos[word[0]] = word[1]

frec_pos = sc.broadcast(frec_pos)

negativas = train_preprocesado.filter( lambda review: review[1]==1 or review[1]==2 or review[1]==3 )
cant_neg = negativas.count()

frec_por_doc_neg = negativas.flatMap(frec_por_doc).reduceByKey( lambda x,y: x+y )
frec_por_doc_neg = frec_por_doc_neg.collect()

frec_neg = {}

for word in frec_por_doc_neg:
    frec_neg[word[0]] = word[1]

frec_neg = sc.broadcast(frec_neg)



# Delta TF-IDF

In [ ]:
import math
from math import sqrt

def delta_tfidf(review_test, review_train, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs):
#def delta_tfidf(review_test, review_train):
    texto = review_test[2]
    frec_word = review_train[3]

    r = 0
    k = 1.2
    # NORMALIZAR
    b = 0.75
    long_doc = len(texto)
    norm = 1 - b + b * (long_doc / promedio_long_docs)

    for word in texto:
        ## TF-IDF
        tf = 0

        if word in frec_word:
            #BM25
            tf = ( (k+1)*frec_word[word] ) / ( frec_word[word] + k*norm )

        idf_pos = 0
        if word in frec_pos.value:
            Pt = frec_pos.value[word]
#            idf_pos = math.log( 1 + (cant_pos+1) / Pt)
            idf_pos = math.log( 1 + (cant_pos) / Pt, 2)


        idf_neg = 0
        if word in frec_neg.value:
            Nt = frec_neg.value[word]
#            idf_neg = math.log( 1 + (cant_neg+1) / Nt)
            idf_neg = math.log( 1 + (cant_neg) / Nt, 2)


        r += tf * ( idf_pos - idf_neg )

#    return (abs(r), review_train[1])
    return (float( abs(r) ), review_train[1])


# Normalizacion datos

In [ ]:
cant_reviews = train_preprocesado.count()

train_long_docs = train_preprocesado.map( lambda review: (1, len(review[2])) ).reduceByKey( lambda x, y: x+y )
long_docs = train_long_docs.collect()
long_docs = long_docs[0][1]
print "long: ", long_docs

promedio_long_docs = float(long_docs) / cant_reviews
print "promedio: ", promedio_long_docs

# LIDERES Y SEGUIDORES

# Lideres

In [ ]:
from math import sqrt

#cant_lideres = int(sqrt(cant_reviews))
cant_lideres = int(sqrt(cant_reviews)/8)
lideres_lista = train_preprocesado.takeSample(False, cant_lideres, seed = 10L)

lideres = sc.broadcast(lideres_lista)

print "cant_leaders: ", len(lideres.value)
print " "
print "lider: ", lideres.value[0]


print "Count: ", train_preprocesado.count()

# Seguidores

In [ ]:

def obtener_lideres_seguidores(seguidor, lideres, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs):
#    lider_de_seguidor = lideres.value[0]
    lider_de_seguidor = lideres.value[0][0]

    max_dist = -1

    for lider in lideres.value:
        dist = delta_tfidf(seguidor, lider, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)
        dist = dist[0]
        if dist > max_dist:
            max_dist = dist
#            lider_de_seguidor = lider
            lider_de_seguidor = lider[0]

    return (lider_de_seguidor, seguidor)

lideres_seguidores = train_preprocesado.map( lambda seguidor: obtener_lideres_seguidores(seguidor, lideres, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs) )

lideres_seguidores = lideres_seguidores.cache()

#for x in lideres_seguidores.take(2):
#    print "lider: ", x[0]
#    print " "
#    print "seguidor: ", x[1]
#    print " "


In [ ]:
"""
aRDD = sc.textFile("a.csv", num_partition)
a_dataframe = pycsv.csvToDataFrame(sqlCtx, aRDD, parseDate=False)
a = a_dataframe.rdd

a_sin_rep = quitar_reviews_repetidos(a)
a_preprocesado = a_sin_rep.map(preprocesar)
a_preprocesado = a_preprocesado.map(bow)

print a_preprocesado.count()

a_lideres_seguidores = a_preprocesado.map( lambda seguidor: obtener_lideres_seguidores(seguidor, lideres, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs) )

print a_lideres_seguidores.count()
#print a_lideres_seguidores.take(1)
"""

In [ ]:

#print lideres_seguidores.getNumPartitions()

print "Count: ", lideres_seguidores.count()

#print lideres_seguidores.map(lambda x: 1).reduce(lambda x,y: x+y)

"""
#seguidor = train_preprocesado.take(2)[1]
seguidor = lideres.value[1]

#lider = lideres.value[0]
#dist = delta_tfidf(seguidor, lider, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)
#print "dist: ", dist
#print " "

a = obtener_lideres_seguidores(seguidor, lideres, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)

#print "a: ", a
print "lider: ", a[0]
print " "
print "seguidor: ", a[1]
print " "
"""

In [ ]:
"""
lider = 85734

seguidores = lideres_seguidores.filter( lambda x: x[0][0]==lider )

print seguidores.getNumPartitions()

for x in seguidores.take(1):
    print "lider: ", x[0]
    print " "
    print "seguidor: ", x[1]
    print " "

#print seguidores.count()
"""

# KNN con Lideres-Seguidores

In [ ]:
lideresRDD = sc.parallelize(lideres_lista, num_partition)

def obtener_dist_a_lideres(review_test, lider, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs):
    dist = delta_tfidf(review_test, lider, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)
    dist = dist[0]

    return (dist, lider)

def obtener_lideres_mas_cercanos(review_test, cant_lideres, lideresRDD, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs):
    dist_lider = lideresRDD.map(lambda lider: obtener_dist_a_lideres(review_test, lider, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs))

    lideres_mas_cercanos = dist_lider.takeOrdered(cant_lideres, key=lambda x: -x[0])

    lideres_cercanos = []
    for lider in lideres_mas_cercanos:
        lideres_cercanos.append(lider[1][0])

    lideres_cercanos = set(lideres_cercanos)
    return lideres_cercanos


In [ ]:
def knn2(review_test, k, lideresRDD, lideres_cercanos, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs):
    cant_lideres = 3
    lideres_cercanos = obtener_lideres_mas_cercanos(review_test, cant_lideres, lideresRDD, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)

    seguidores = lideres_seguidores.filter( lambda lider: lider[0] in lideres_cercanos )

    dist_seguidores = seguidores.map(lambda seguidor: delta_tfidf(review_test, seguidor[1], frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs))

    k_mas_cercanos = dist_seguidores.takeOrdered(k, key=lambda x: -x[0])

#    k_mas_cercanos = lideres_seguidores.filter( lambda lider: lider[0] in lideres_cercanos )\
#                                        .map(lambda seguidor: delta_tfidf(review_test, seguidor[1], frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs))\
#                                        .takeOrdered(k, key=lambda x: -x[0])

    return k_mas_cercanos


# Predecir uno

In [ ]:
## PREDECIR UNO SOLO

from timeit import default_timer as timer
#import time

def predecir(k_mas_cercanos):
    prediccion_total = 0
    for cercano in k_mas_cercanos:
        prediccion_total += cercano[1]

    return float(prediccion_total) / len(k_mas_cercanos)
    ## REDONDEANDO
#    return prediccion_total / len(k_mas_cercanos)


reviewRDD = sc.textFile("review")
review_dataframe = pycsv.csvToDataFrame(sqlCtx, reviewRDD, parseDate=False)
reviews = review_dataframe.rdd

reviews_preprocesado = reviews.map(preprocesar).map(bow)
test_pred = reviews_preprocesado.collect()
test_pred = test_pred[0]

print "TEST_PRED: ", test_pred[0:2]

cant_lideres = 1
lideres_cercanos = obtener_lideres_mas_cercanos(test_pred, cant_lideres, lideresRDD, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)

print "lideres_cercanos: ", lideres_cercanos

k = 3
"""
k_mas_cercanos = lideres_seguidores.filter( lambda lider: lider[0] in lideres_cercanos )\
                                    .map(lambda seguidor: delta_tfidf(test_pred, seguidor[1], frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs))\
                                    .takeOrdered(k, key=lambda x: -x[0])
"""

k_mas_cercanos = knn2(test_pred, k, lideresRDD, lideres_cercanos, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)

print k_mas_cercanos


# Predecir varios

In [ ]:
## PREDECIR 10

from math import sqrt
from timeit import default_timer as timer
#import time

def predecir(k_mas_cercanos):
    prediccion_total = 0
    for cercano in k_mas_cercanos:
        prediccion_total += cercano[1]

    return float(prediccion_total) / len(k_mas_cercanos)
    ## REDONDEANDO
#    return prediccion_total / len(k_mas_cercanos)

## 100
testRDD = sc.textFile('data_test.csv', num_partition)
test_dataframe = pycsv.csvToDataFrame(sqlCtx, testRDD, parseDate=False)
test = test_dataframe.rdd
test_sin_rep = quitar_reviews_repetidos(test)
test_preprocesado = test_sin_rep.map(preprocesar)

reviews = test_preprocesado.take(100)
cant_ej = 100

## 10
"""
reviewRDD = sc.textFile("reviews_test")
review_dataframe = pycsv.csvToDataFrame(sqlCtx, reviewRDD, parseDate=False)
reviews = review_dataframe.rdd
reviews_preprocesado = reviews.map(preprocesar)

reviews = reviews_preprocesado.collect()
cant_ej = 10
"""

#test = test_preprocesado.take(cant_ej)

#for t in test:
#    print "t: ", t

start = timer()

## PREDECIR VARIOS
error = 0
correct = 0
for test_pred in reviews:
    print "TEST_PRED: ", test_pred[0:2]

    lideres_cercanos = obtener_lideres_mas_cercanos(test_pred, 1, lideresRDD, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)

    k_mas_cercanos = knn2(test_pred, 3, lideresRDD, lideres_cercanos, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)
    print "k_mas_cercanos: ", k_mas_cercanos

    prediccion = predecir(k_mas_cercanos)

    print "PRED: ", prediccion
    print " "

    neg = prediccion <= 3.5 and test_pred[1] <= 3.5
    pos = prediccion > 3.5 and test_pred[1] > 3.5
    if neg or pos:
        correct += 1
    error += (prediccion - test_pred[1])**2

error = sqrt(error)
print "ERROR: ", error

print "accuracy: ", ( float(correct)/cant_ej ) * 100


end = timer()
print "time: ", (end-start)

#p = float(correct) / cant_ej
#r = float(correct) / train_preprocesado.count()
#print "F1 score: ", ( 2*p*r / (p+r) )


# Predecir test

In [ ]:
def preprocesar_test(review):
    texto = quitar_signos(review.Text)
    resumen = quitar_signos(review.Summary)

#    texto = texto.lower()
#    resumen = resumen.lower()

    texto_palabras = texto.split(" ")
    resumen_palabras = resumen.split(" ")

#    texto_palabras = texto_palabras + ngrams(texto_palabras, 2)
#    resumen_palabras = resumen_palabras + ngrams(resumen_palabras, 2)

    texto_palabras = quitar_stopWords(texto_palabras)
    resumen_palabras = quitar_stopWords(resumen_palabras)

    estructura = [review.Id, -1, resumen_palabras+texto_palabras]

    return estructura


In [ ]:

from math import sqrt
from timeit import default_timer as timer
#import time

def predecir(k_mas_cercanos):
    prediccion_total = 0
    for cercano in k_mas_cercanos:
        prediccion_total += cercano[1]

    return float(prediccion_total) / len(k_mas_cercanos)


reviewRDD = sc.textFile("test.csv")
review_dataframe = pycsv.csvToDataFrame(sqlCtx, reviewRDD, parseDate=False)
reviews = review_dataframe.rdd

reviews_preprocesado = reviews.map(preprocesar_test)
reviews = reviews_preprocesado.collect()


In [ ]:
try:
    predic = open("test_predicciones.txt", 'w')
except IOError as exc:
    print "Error al abrir el archivo de entrada, más información: ", exc

start = timer()

cant_ej = 0

error = 0
correct = 0
for test_pred in reviews:
    cant_ej += 1

    lideres_cercanos = obtener_lideres_mas_cercanos(test_pred, 1, lideresRDD, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)

    k_mas_cercanos = knn2(test_pred, 3, lideresRDD, lideres_cercanos, frec_pos, cant_pos, frec_neg, cant_neg, promedio_long_docs)

    prediccion = predecir(k_mas_cercanos)

    neg = prediccion <= 3.5 and test_pred[1] <= 3.5
    pos = prediccion > 3.5 and test_pred[1] > 3.5
    if neg or pos:
        correct += 1
    error += (prediccion - test_pred[1])**2

    if cant_ej % 10 == 0:
        print "TEST_PRED: ", test_pred[0:2]
        print "k_mas_cercanos: ", k_mas_cercanos
        print "PRED: ", prediccion
        print "cant_ej: ", cant_ej
        print " "

    predic.write(str(prediccion) + " " + str(test_pred[0]))


predic.close()
error = sqrt(error)
print "ERROR: ", error

print "accuracy: ", ( float(correct)/cant_ej ) * 100

end = timer()
print "time: ", (end-start)